In [1]:
%%file requirements.txt
datasets==1.16.1
pycountry==20.7.3
wandb==0.12.7
sentence-transformers==2.1.0
rapidfuzz==1.8.3

Writing requirements.txt


In [2]:
pip install -qr requirements.txt

     |████████████████████████████████| 298 kB 4.3 MB/s 
     |████████████████████████████████| 10.1 MB 20.3 MB/s 
     |████████████████████████████████| 1.7 MB 32.9 MB/s 
     |████████████████████████████████| 78 kB 5.6 MB/s 
     |████████████████████████████████| 854 kB 46.1 MB/s 
     |████████████████████████████████| 1.1 MB 45.6 MB/s 
     |████████████████████████████████| 61 kB 520 kB/s 
     |████████████████████████████████| 243 kB 46.2 MB/s 
     |████████████████████████████████| 132 kB 33.7 MB/s 
     |████████████████████████████████| 180 kB 58.1 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 140 kB 40.9 MB/s 
     |████████████████████████████████| 3.1 MB 46.0 MB/s 
     |████████████████████████████████| 3.3 MB 34.6 MB/s 
     |████████████████████████████████| 1.2 MB 23.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 596 kB 42.9 MB/s 
     |████████████████

In [3]:
import traceback
import warnings
from typing import List

import numpy as np
import pandas as pd
import pycountry
import torch
import wandb
from datasets import load_dataset
from tqdm.auto import tqdm
from sentence_transformers import util

# Embedding

In [4]:
# initialize

In [5]:
from rapidfuzz import process

In [6]:
# embed


# Execution

In [7]:
config = wandb.config

config.embedder = "levenshtein"
config.name = f"tapaco-{config.embedder}"

In [8]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
wandb.init(
      entity="eenlp",
      project="paraphrase_detection",
      job_type=f"create-dataset",
      name=f"create-tapaco-{config.embedder}"
)

wandb: Currently logged in as: matts (use `wandb login --relogin` to force relogin)


In [ ]:
# DATASET_ONLY_FIRST_N = 20_000
DATASET_ONLY_FIRST_N = 10_000

config.DATASET_ONLY_FIRST_N = DATASET_ONLY_FIRST_N

dataset_name = "tapaco"

dataset = load_dataset("tapaco", "all_languages")
df = dataset["train"].to_pandas()

df["paraphrase_set_id"] = df["paraphrase_set_id"].astype(int)

languages_to_keep = [
    "be",  # Belarusian
    "bg",  # Bulgarian
    "cs",  # Czech
    "en",  # English
    "et",  # Estonian
    "hr",  # Croatian
    "hu",  # Hungarian
    "hy",  # Armenian
    "lt",  # Lithuanian
    "mk",  # Macedonian
    "pl",  # Polish
    "ro",  # Romanian
    "ru",  # Russian
    "sl",  # Slovenian
    "sr",  # Serbian
    "uk",  # Ukrainian
]

df = df[df["language"].isin(languages_to_keep)]
assert len(languages_to_keep) == len(
    df["language"].unique()
), "Count of filtered languages doesn't match, probably a typo in 'languages_to_keep'?"

# paraphrase_set_id = 0 seems to mean "unassigned".
df = df[df["paraphrase_set_id"] != 0].copy()

# We need to generate example pairs somehow.
# As of the current version of the code, 50% of the generated examples are true paraphrases (label = 1),
# 25% are chosen from the most similar non-paraphrases (label = 0),
# and 25% is random negative examples (label = 0).

result = []
np.random.seed(0)
for language, df_language in tqdm(df.groupby("language"), "language", position=0):
    try:
        language_name = pycountry.languages.get(alpha_2=language).name

        df_language = df_language[:DATASET_ONLY_FIRST_N].copy()

        for paraphrase_set_id, df_set in tqdm(
            df_language.groupby("paraphrase_set_id"),
            "paraphrase_set_id",
            position=1,
        ):
            if len(df_set) <= 1:
                continue

            df_negatives = df_language[
                df_language["paraphrase_set_id"] != paraphrase_set_id
            ]

            for row in df_set.itertuples():
                result.append(
                    {
                        "sentence1": row.paraphrase,
                        "sentence2": np.random.choice(
                            df_set[df_set.index != row.Index]["paraphrase"]
                        ),
                        "label": 1,
                        "lang": language_name,
                        "tapaco_negative_type": 0,
                    }
                )

                # TODO this can sample the same pair twice, consider fixing it
                result.append(
                    {
                        "sentence1": row.paraphrase,
                        "sentence2": np.random.choice(
                            df_set[df_set.index != row.Index]["paraphrase"]
                        ),
                        "label": 1,
                        "lang": language_name,
                        "tapaco_negative_type": 0,
                    }
                )

                # similar negative
                top_results = process.extract(
                    row.paraphrase,
                    df_negatives["paraphrase"],
                    limit=1,
                )

                result.append(
                    {
                        "sentence1": row.paraphrase,
                        "sentence2": top_results[0][0],
                        "label": 0,
                        "lang": language_name,
                        "tapaco_negative_type": 1,
                    }
                )

                # random negative
                result.append(
                    {
                        "sentence1": row.paraphrase,
                        "sentence2": np.random.choice(df_negatives["paraphrase"]),
                        "label": 0,
                        "lang": language_name,
                        "tapaco_negative_type": 2,
                    }
                )
    except KeyboardInterrupt:
        raise
    # except:
    #     warnings.warn(traceback.format_exc())
    #     continue

result = pd.DataFrame(result)
result["source"] = dataset_name
result["split"] = "train"

wandb.Table.MAX_ARTIFACT_ROWS = 500_000
artifact = wandb.Artifact(name=config.name, type="tapaco-dataset")
artifact.add(wandb.Table(dataframe=result), "data")
wandb.run.log_artifact(artifact)

Downloading:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.69k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/32.2M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

paraphrase_set_id:   0%|          | 0/940 [00:00<?, ?it/s]

paraphrase_set_id:   0%|          | 0/1942 [00:00<?, ?it/s]

paraphrase_set_id:   0%|          | 0/309 [00:00<?, ?it/s]

paraphrase_set_id:   0%|          | 0/3236 [00:00<?, ?it/s]

paraphrase_set_id:   0%|          | 0/2699 [00:00<?, ?it/s]

In [ ]:
wandb.run.finish()